In [6]:
from sympy import *
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import itertools
import random
#import sage

In [7]:
# A, b, c
A = np.array([[1,0,1,0,1,0,-1,0,0,0],[0,1,0,1,0,1,0,-1,0,0],[0,0,2,1,0,0,0,0,1,0],[0,0,1,2,0,0,0,0,0,1]])
c = np.array([35,40,16,19,47,54,0,0,0,0])

In [26]:
Matrix(A)

Matrix([
[1, 0, 1, 0, 1, 0, -1,  0, 0, 0],
[0, 1, 0, 1, 0, 1,  0, -1, 0, 0],
[0, 0, 2, 1, 0, 0,  0,  0, 1, 0],
[0, 0, 1, 2, 0, 0,  0,  0, 0, 1]])

# Pre

In [8]:
def to_polynomial(coef,vars):
            '''
            Function to define a single row of the coefficient as a polynomial
            '''
            res1 = 1
            res2 = 1
            for i in range(len(coef)):
                if coef[i] > 0:
                    res1 = res1*vars[i]**coef[i]
                elif coef[i] < 0:
                    res2 = res2*vars[i]**(-coef[i])
            res = res1 - res2
            return res

In [9]:
def monomial(p):
    return [prod(x**k for x, k in zip(p.gens, mon)) for mon in p.monoms()]

# Main

In [28]:
def gen(A):
    # column/row dimension of a matrix
    r = A.shape[0]
    n = A.shape[1]
    A = np.array(A)
    
    # Define symbolic variables xs for each column (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)
    
    # Calculate a basis for the kernel
    def Ker(A):
        A = Matrix(A)
        B = np.array(Matrix(A).nullspace()).transpose()
        C = Matrix(B[0]).transpose()
        return C
    
    # find an equivalent basis with all base vectors lying in the same orthant
    def same_orthant(A):    
        # compute the highest value in abstract value in a numpy natrix
        def abs_max(A):
            A = np.array(A)
            return abs(max(A.min(), A.max(), key=abs))

        for i in range(1,r):
            A[i,:] = A[i,:]+A[i-1,:]*(1+abs_max(A[i,:]))
        A_fin = np.array(A)
        A_fin = A_fin.astype(int)
        A_fin = Matrix(A_fin)
        return A_fin
    
    # compute J and prepare the function phi & phi_inv
    def nega_index(A):
        J = [ A[0,k]<0 for k in range(0,A.shape[1])]
        return J

    def reverse_sign(A):
        n = A.shape[1]
        A = np.array(A)
        for i in range(0,n):
            if nega_index(A)[i]:
                A[:,i] = - A[:,i]
        A_fin = A.astype(int)
        A_fin = Matrix(A_fin)
        return A_fin

    def phi(A):
        G_J=[]
        for k in range(0,A.shape[0]):
            G_J.append(to_polynomial(A[k,:],xs))
        return G_J

    def phi_inv(G_J):
        G_J_len = len(G_J)

        temp = []
        vp = [0]*n
        vm = [0]*n
        for k in range(0,G_J_len):
            for i in range(0,n):
                p = monomial(Poly(G_J[k]))[0]
                m = monomial(Poly(G_J[k]))[1]
                vp[i] = degree(p,xs[i])
                vm[i] = degree(m,xs[i])
            v = np.array(vp) - np.array(vm)
            v = v.astype(int)
            temp.append(v)
            temp_fin = Matrix(temp)
        return temp_fin, temp
    
    
    def TJ_operation(G_J,J):
        for j in J:
            if j:
                # change position of xs[j]
                xs_list = list(xs)
                index = J.index(j)
                J[index] = False
                temp = xs_list[index]
                xs_list.remove(temp)
                xs_list.insert(0,temp)

                G_J = list(groebner(G_J, xs_list , order='lex'))

                A3, _ = phi_inv(G_J)
                A3[:,index] = - A3[:,index]
                G_J = phi(A3)

        return G_J
    
   
    # MAIN
    # generating set for the corresponding matrix kernel
    A1 = Ker(A)
    
    # matrix pre-operation
    A2 = same_orthant(A1)
    J = nega_index(A2)
    A3 = reverse_sign(A2)
    
    # groebner basis T_J operation
    G_J = phi(A3)
    G_zero = TJ_operation(G_J,J)
    return G_zero

In [29]:
def groeb(G_zero,c):
    # column/row dimension of a matrix
    n = c.shape[0]
    
    # Define symbolic variables xs for each column (index 0 in Python)
    sym_str_x = 'x:' + str(A.shape[1])
    xs = symbols(sym_str_x)

    def phi_inv(G_J):
        G_J_len = len(G_J)

        temp = []
        vp = [0]*n
        vm = [0]*n
        for k in range(0,G_J_len):
            for i in range(0,n):
                p = monomial(Poly(G_J[k]))[0]
                m = monomial(Poly(G_J[k]))[1]
                vp[i] = degree(p,xs[i])
                vm[i] = degree(m,xs[i])
            v = np.array(vp) - np.array(vm)
            v = v.astype(int)
            temp.append(v)
            temp_fin = Matrix(temp)
        return temp_fin, temp

    # define new order in Sympy
    class compositeOrder(polys.orderings.MonomialOrder):
        """Composite-Lexicographic order of monomials. """
        alias = 'clex'
        is_global = True

        def __call__(self, monomial):
            return (np.dot(c,np.array(monomial)),monomial)
    polys.orderings.__all__.append('clex')
    clex = compositeOrder()
    polys.orderings._monomial_key['clex'] = compositeOrder() 
    
    # apply groebner basis algorithm 
    G_fin = groebner(G_zero,xs,order='clex')
    _ , G_fin = phi_inv(list(G_fin))
    return G_fin

In [30]:
#temp = gen(A,c)
#groeb(temp,c)

# Experiment

In [31]:
def augmentation(z_feas,c,T):
    # z_feaas: feasible point ; c: cost; T: universal test set
    exist_aug = True
    i = 0
    while exist_aug:
        exist_aug = False
        for t in T:
            if np.dot(c, t, out=None)>0 and np. all((z_feas-t>=0)):
                z_feas = z_feas-t
                i = 1+i
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
            if np.dot(c, t, out=None)<0 and np. all((z_feas+t>=0)):
                z_feas = z_feas+t
                i = i+1
                #print('Iteration step', i,': vector', z_feas)
                exist_aug = True
    #print('Achieve an optimum!')
    return z_feas

In [32]:
def feasible_solu(b):
    return [0,0,0,0,b[0],b[1],0,0,0,0]

In [35]:
T_1 = gen(A)


In [36]:
#T
opt = []
for i in range(0,1000,3):
    b = [300+i,300+i,200+10*i,200+10*i]
    temp = augmentation(feasible_solu(b),c,groeb(T_1,c))
    opt.append(temp)

opt

[array([300, 300,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([303, 303,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([306, 306,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([309, 309,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([312, 312,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([315, 315,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([318, 318,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([321, 321,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([324, 324,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([327, 327,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([330, 330,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([333, 333,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([336, 336,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([339, 339,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([342, 342,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([345, 345,   0,   0,   0,   0,   0,   0,   0,   0]),
 array([348, 348,   0,   0,   0,   0,   